# Inverse matrix descriptor in TensorFlow

This notebook explains how to make an inverse matrix descriptor in TensorFlow starting from a matrix of `[n_samples, n_features]`. Each row is a different configuration, and each atom has 3 cartesian coordinates x, y and z (so, `n_features` is a multiple of 3).

The first step is to generate a tensor. In this case we have a system with 3 atoms.

In [1]:
import tensorflow as tf
import numpy as np

xyz = tf.constant([[1., 2., 3., 4., 5., 6., 1, 1, 1, 2, 2, 2], [2, 4, 6, 8, 10,12, 1, 1, 1, 2, 2, 2], [3, 3, 3, 4, 4, 4, 1, 1, 1, 2, 2, 2]])
n_atoms = 4
n_samples = 3
xyz_3d = tf.reshape(xyz, shape=(n_samples, n_atoms, 3))

sess = tf.Session()
sess.run(xyz_3d)

array([[[  1.,   2.,   3.],
        [  4.,   5.,   6.],
        [  1.,   1.,   1.],
        [  2.,   2.,   2.]],

       [[  2.,   4.,   6.],
        [  8.,  10.,  12.],
        [  1.,   1.,   1.],
        [  2.,   2.,   2.]],

       [[  3.,   3.,   3.],
        [  4.,   4.,   4.],
        [  1.,   1.,   1.],
        [  2.,   2.,   2.]]], dtype=float32)

The next step involves expanding the dimensions of `xyz_3d` in order to exploit [broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html). This is quite confusing.

In [2]:
expanded_a = tf.expand_dims(xyz_3d, 2)
expanded_b = tf.expand_dims(xyz_3d, 1)

print(sess.run(tf.shape(expanded_a)))
print(sess.run(tf.shape(expanded_b)))

[3 4 1 3]
[3 1 4 3]


The next step is to take the difference squared of these two expanded matrices. For each column in the 3 matrices in `xyz_3d`, this function creates a matrix with shape `[n_atoms, n_atoms]`. So, for the first column of the first matrix of `xyz_3d`, it gives this (call this matrix *diffA* for reference):

|       | 1 | 4 | 1 | 2 |
| :     | : | : | : | : |
| **1** | 0 | 9 | 0 | 1 |
| **4** | 9 | 0 | 9 | 4 |
| **1** | 0 | 9 | 0 | 1 |
| **2** | 1 | 4 | 1 | 0 |

For the second column it gives this:

|       | 2 | 5  | 1  | 2 |
| :     | : | :  | :  | : |
| **2** | 0 | 9  | 1  | 0 |
| **5** | 9 | 0  | 16 | 9 |
| **1** | 1 | 16 | 0  | 1 |
| **2** | 0 | 9  | 1  | 0 |


And so on and so forth.

However, because dimensions want to be confusing, these matrices appear as columns. So, the first row of *diffA* is the first column of the first matrix in `diff2`, the second row of *diffA* is the first column of the second matrix in `diff2`.



In [3]:
diff2 = tf.squared_difference(expanded_a, expanded_b)
print(sess.run(diff2))

[[[[   0.    0.    0.]
   [   9.    9.    9.]
   [   0.    1.    4.]
   [   1.    0.    1.]]

  [[   9.    9.    9.]
   [   0.    0.    0.]
   [   9.   16.   25.]
   [   4.    9.   16.]]

  [[   0.    1.    4.]
   [   9.   16.   25.]
   [   0.    0.    0.]
   [   1.    1.    1.]]

  [[   1.    0.    1.]
   [   4.    9.   16.]
   [   1.    1.    1.]
   [   0.    0.    0.]]]


 [[[   0.    0.    0.]
   [  36.   36.   36.]
   [   1.    9.   25.]
   [   0.    4.   16.]]

  [[  36.   36.   36.]
   [   0.    0.    0.]
   [  49.   81.  121.]
   [  36.   64.  100.]]

  [[   1.    9.   25.]
   [  49.   81.  121.]
   [   0.    0.    0.]
   [   1.    1.    1.]]

  [[   0.    4.   16.]
   [  36.   64.  100.]
   [   1.    1.    1.]
   [   0.    0.    0.]]]


 [[[   0.    0.    0.]
   [   1.    1.    1.]
   [   4.    4.    4.]
   [   1.    1.    1.]]

  [[   1.    1.    1.]
   [   0.    0.    0.]
   [   9.    9.    9.]
   [   4.    4.    4.]]

  [[   4.    4.    4.]
   [   9.    9.    9.]
   [   0. 

Now, we sum the columns of the matrix above. The 3 column vectors are joined together in a matrix. This is because the distance between two atoms is:

`(x1 - x2)^2 + (y1 - y2)^2 + (z1 - z2)^2`

in `diff2` one calculates all the possible pair of distances for the x, then for the y and then for the z. This is why they have to be summed together to give the total distance.


In [4]:
diff2_sum = tf.reduce_sum(diff2, axis=3)
diff_sum = tf.sqrt(diff2_sum)

Now that we have the distance matrix for each configuration of the atoms, we can take the strictly upper part of the matrix (since it's a symmetric matrix, there is no need to keep it all). 

In [5]:
ones = tf.ones_like(diff_sum)
mask_a = tf.matrix_band_part(ones, 0, -1)
mask_b = tf.matrix_band_part(ones, 0, 0)
mask = tf.cast(mask_a - mask_b, dtype=tf.bool) # Transfoorm into bool

upper_triangular_conc = tf.boolean_mask(diff_sum, mask)
upper_triangular = tf.reshape(upper_triangular_conc, shape=(n_samples, int(n_atoms * (n_atoms-1) * 0.5)))
print(sess.run(upper_triangular))

[[  5.19615221   2.23606801   1.41421342   7.07106733   5.38516474
    1.73205078]
 [ 10.39230442   5.91607952   4.47213602  15.84297943  14.14213467
    1.73205078]
 [  1.73205078   3.46410155   1.73205078   5.19615221   3.46410155
    1.73205078]]


For 4 atoms, the distance matrix is 4x4. The upper triangular (excluding the diagonal) contains only 6 elements. 